In [1]:
import numpy as np
import cv2
import pandas as pd

import torch
from torch import nn


from scipy.io import wavfile

from torchsummary import summary
from torchvision import models

from tqdm import tqdm

import os
import shutil
import tarfile

import librosa
import random

import pickle

import time

import IPython.display as ipd

import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
import seaborn as sns

%matplotlib inline

In [2]:
def get_paths_to_wavs(path_to_dataset):
    file_paths_list = []

    for root, dirs, files in os.walk(path_to_dataset):
        if len(files) != 0:
            file_paths_list += [os.path.join(root, f) for f in files if f.endswith('.wav')]

    return file_paths_list

def get_paths_to_npys(path_to_dataset):
    # get a list with all absolute paths to each file
    file_paths_list = []

    for root, dirs, files in os.walk(path_to_dataset):
        if len(files) != 0:
            file_paths_list += [os.path.join(root, f) for f in files if f.endswith('.npy')]
            #file_paths_list += [os.path.join(root, f) for f in files if os.path.isdir(os.path.join(root, f))]

    return file_paths_list

class numpy_ravdess_dataset(torch.utils.data.Dataset):
    '''
    Due to librosa reads wav-files very slow it is more preferable to read the
    numpy representations of the original wavs
    '''

    emotions_dict = {
        0: 'neutral',
        1: 'calm',
        2: 'happy',
        3: 'sad',
        4: 'angry',
        5: 'fearful',
        6: 'disgust',
        7: 'surprised'
        }

    def __init__(self, paths_to_wavs_list, spectrogram_shape, mode):
        super(numpy_ravdess_dataset, self).__init__()

        self.paths_to_wavs_list = paths_to_wavs_list

        self.mfcc_rows = spectrogram_shape[0]
        self.mfcc_cols = spectrogram_shape[1]
        self.mode = mode

    def __len__(self):
        return len(self.paths_to_wavs_list)
    '''
    def read_audio(self, path_to_wav):
        return np.load(path_to_wav, allow_pickle=True)
    '''
    def read_audio(self, path_to_wav):
        sr, wav = wavfile.read(path_to_wav)
        wav = (wav / 32768).astype(np.float32)
        return wav, sr

    def get_class_label(self, path_to_file):
        # Parse the filename, which has the following pattern:
        # modality-vocal_channel-emotion-intensity-statement-repetition-actor.wav
        # e.g., '02-01-06-01-02-01-12.wav'
        file_name = os.path.split(path_to_file)[1]
        file_name = file_name[:-4]
        class_label = int(file_name.split('-')[2]) - 1 # 2 is a number of emotion code
        return class_label
        

    def __getitem__(self, idx):
        path_to_wav = self.paths_to_wavs_list[idx]
        # debug
        #print(path_to_wav)

        # read the wav file
        wav, sr = self.read_audio(path_to_wav)       

        # augmentation
        
        if self.mode == 'TRAIN':
            # add noise
            if np.random.randint(0, 2) == 1:
                sigma = np.random.uniform(0.0009, 0.0051)
                noise = sigma * np.random.randn(len(wav))
                wav += noise
            # stretch wav
            if np.random.randint(0, 2) == 1:
                factor = np.random.uniform(0.5, 1.2)
                wav = librosa.effects.time_stretch(wav, 2)
            # change pitch
            if np.random.randint(0, 2) == 1:
                factor = np.random.uniform(-1.5, 1.1)
                wav = librosa.effects.pitch_shift(wav, sr=sr, n_steps=factor)
    
        # get mfcc coefficients
        #mfccs = librosa.feature.mfcc(wav, sr=sr, n_mfcc=self.mfcc_rows, n_mels=self.mfcc_rows).astype(np.float32)
        '''
        if self.mode == 'TRAIN':
            # augment by choosing n_fft
            n_fft_list = [i for i in range(1024, 2049, 32)]
            idx = np.random.randint(len(n_fft_list))
            n_fft = n_fft_list[idx]
        else:
            n_fft = 2048

        '''
        '''
        n_fft = 2048
        mfccs = librosa.feature.melspectrogram(wav, sr=sr, n_mels=self.mfcc_rows, n_fft=n_fft, hop_length=128).astype(np.float32)

        '''
        mfccs = librosa.core.stft(wav, n_fft=512)#.astype(np.float32)
        mfccs = np.abs(mfccs)**2
        mfccs = np.log(mfccs + 0.1)
        mfccs = mfccs[:-1]
        
        #mfccs = (mfccs - mfccs.mean())/np.std(mfccs)

        actual_mfcc_cols = mfccs.shape[1]

        # prmitive time-shifting augmentation
        target_real_diff = actual_mfcc_cols - self.mfcc_cols
        if target_real_diff > 0:
            
            if self.mode == 'TRAIN':
                beginning_col = np.random.randint(target_real_diff)
            else:
                beginning_col = actual_mfcc_cols//2 - self.mfcc_cols//2

            mfccs = mfccs[:, beginning_col:beginning_col + self.mfcc_cols]
            #mfccs = mfccs[:, beginning_col:beginning_col + self.mfcc_cols]

        elif target_real_diff < 0:
            zeros = np.zeros((self.mfcc_rows, self.mfcc_cols), dtype=np.float32)
            
            if self.mode == 'TRAIN':
                beginning_col = np.random.randint(self.mfcc_cols-actual_mfcc_cols)
            else:
            
                beginning_col = self.mfcc_cols//2 - actual_mfcc_cols//2
            zeros[..., beginning_col:beginning_col+actual_mfcc_cols] = mfccs
            #zeros[..., beginning_col:beginning_col+actual_mfcc_cols] = mfccs
            mfccs = zeros
            #mfccs = np.pad(mfccs, ((0, 0), (0, np.abs(target_real_diff))), constant_values=(0), mode='constant')

        # make the data compatible to pytorch 1-channel CNNs format
        mfccs = np.expand_dims(mfccs, axis=0)

        # normalize spectrogram
        mfccs = (mfccs - mfccs.mean())/np.std(mfccs)

        # Parse the filename, which has the following pattern:
        # modality-vocal_channel-emotion-intensity-statement-repetition-actor.wav
        # e.g., '02-01-06-01-02-01-12.wav'
        #file_name = os.path.split(path_to_wav)[1]
        #file_name = file_name[:-4]
        #class_label = int(file_name.split('-')[2]) - 1 # 2 is a number of emotion code
        #class_label = np.array(class_label)
        class_label = self.get_class_label(path_to_wav)
        # !!!!!!!!!
        return torch.from_numpy(mfccs), class_label#, path_to_wav

class numpy_crema_dataset(numpy_ravdess_dataset):
    emotions_dict = {
        'ANG': 0,
        'DIS': 1,
        'FEA': 2,
        'SAD': 3,
        'HAP': 4,
        'NEU': 5
    }

    label2str = {
        0: 'ANG',
        1: 'DIS',
        2: 'FEA',
        3: 'SAD',
        4: 'HAP',
        5: 'NEU'
    }
    
    def get_class_label(self, path_to_file):
        file_name = os.path.split(path_to_file)[1]
        file_name = file_name[:-4]
        emotion_name = file_name.split('_')[2] # 2 is a number of emotion code
        return self.emotions_dict[emotion_name]

class numpy_iemocap_dataset(numpy_ravdess_dataset):
    '''
    emotions_dict = {
        'exc': 0,
        'sad': 1,
        'fru': 2,
        'hap': 3,
        'neu': 4,
        'sur': 5,
        'ang': 6,
        'fea': 7,
        'dis': 8,
        #'oth': 9
    }
    '''
    emotions_dict = {
        'exc': 0,
        'sad': 1,
        'fru': 2,
        'hap': 3,
        'neu': 4,
        'ang': 5,
    }

    def get_class_label(self, path_to_file):
        file_name = os.path.split(path_to_file)[1]
        file_name = file_name[:-4]
        emotion_name = file_name.split('_')[-1] # the last is a position of emotion code
        return self.emotions_dict[emotion_name]

class crema_gender_dataset(numpy_ravdess_dataset):
    emotions_dict = {
        'ANG_Male': 0,
        'DIS_Male': 1,
        'FEA_Male': 2,
        'SAD_Male': 3,
        'HAP_Male': 4,
        'NEU_Male': 5,
        'ANG_Female': 6,
        'DIS_Female': 7,
        'FEA_Female': 8,
        'SAD_Female': 9,
        'HAP_Female': 10,
        'NEU_Female': 11
    }

    label2str = {
        0: 'ANG',
        1: 'DIS',
        2: 'FEA',
        3: 'SAD',
        4: 'HAP',
        5: 'NEU'
    }
    def __init__(self, paths_to_wavs_list, spectrogram_shape, mode, gender_df):
        super().__init__(paths_to_wavs_list, spectrogram_shape, mode)
        self.gender_df = gender_df
    
    def get_class_label(self, path_to_file):
        
        file_name = os.path.split(path_to_file)[1]
        file_name = file_name[:-4]
        name_list = file_name.split('_') # 2 is a number of emotion code
        emotion_name = name_list[2]
        actor_id = int(name_list[0])

        gender = self.gender_df[self.gender_df['ActorID'] == actor_id]['Sex'].values[0]

        return self.emotions_dict['{}_{}'.format(emotion_name, gender)]

In [4]:
def index_dataset(paths_list):
    human_id_dict = {}# OrderedDict()
    phrase_dict = {}#OrderedDict()
    emotion_dict = {}#OrderedDict()
    
    for path in paths_list:
        file_name = os.path.split(path)[1]
        file_name = file_name[:-4]
        name_list = file_name.split('_') # 2 is a number of emotion code
        human_id = name_list[0]
        phrase_id = name_list[1]
        emotion_name = name_list[2]
        
        try:
            human_id_dict[human_id] += 1
        except KeyError:
            human_id_dict[human_id] = 1

        try:
            phrase_dict[phrase_id] += 1
        except KeyError:
            phrase_dict[phrase_id] = 1

        try:
            emotion_dict[emotion_name] += 1
        except KeyError:
            emotion_dict[emotion_name] = 1

    for key in emotion_dict:
        emotion_dict[key] /= len(paths_list)

    for key in phrase_dict:
        phrase_dict[key] /= len(paths_list)

    for key in human_id_dict:
        human_id_dict[key] /= len(paths_list)

    return human_id_dict, phrase_dict, emotion_dict

def validate(model, criterion, testloader, device):

    dataset_size = len(testloader.dataset)  
        
    correct = 0
    total = 0

    model.eval()

    epoch_loss = 0.0
    
    for i, (data, target) in enumerate(testloader):
        t0 = time.time()
        data = data.to(device)
        target = target.to(device)
        
        with torch.no_grad():
            # run forward step
            predicted = model(data)

            loss = criterion(predicted, target)

            epoch_loss += loss.item() * data.size(0)

        _, pred_labels = torch.max(predicted.data, 1)

        total += target.size(0)
        correct += (pred_labels == target).sum().item()


    return epoch_loss/dataset_size, correct/total


def train_num_epochs(model, trainloader, testloader, device, criterion, optimizer, starting_epoch, ending_epoch, basic_name, path_to_weights):
    '''
    model - neural network
    trainloader - pytorch dataloader for training set
    testloader - pytorch dataloader for test set
    device - cpu / cuda
    criterion - loss function (nn.CrossEntropyLoss())
    optimizer - (Adam)
    starting_epoch - 
    ending_epoch - 
    '''
    dataset_size = len(trainloader.dataset)  

    correct = 0
    total = 0

    best_acc = 0.0

    train_acc_list = []
    val_acc_list = []
    train_loss_list = []
    val_loss_list = []

    # iterate over epochs
    for epoch_num in range(starting_epoch, ending_epoch):
        print('Epoch #%d' % (epoch_num))

        # iterate over batches
        epoch_loss = 0.0

        model.train()

        t0 = time.time()
        for i, (data, target) in enumerate(trainloader):
            
            data = data.to(device)
            target = target.to(device)

            # zero all the gradient tensors
            optimizer.zero_grad()
            # run forward step
            predicted = model(data)

            # compute loss
            loss = criterion(predicted, target)

            # compute gradient tensors
            loss.backward()

            # update parameters
            optimizer.step()

            # compute the loss value
            epoch_loss += loss.item() * data.size(0)
            
            total += target.size(0)
            _, pred_labels = torch.max(predicted.data, 1)

            correct += (pred_labels == target).sum().item()
        
        t = time.time() - t0
            
        
        epoch_loss /=  dataset_size
        train_acc = correct/total
        print('# Time passed: %.0f s' % (t))
        print('# Epoch loss = %.4f' % (epoch_loss))
        print('# Train acc = {}'.format(train_acc))
        print('# Validation process on validation set')
        val_loss, val_acc = validate(model, criterion, testloader, device)
        print('# Validation loss = {}'.format(val_loss))
        print('# Validation acc = {}'.format(val_acc))

        #print(val_acc > best_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            model_name = basic_name + '_ep-{}_loss-{:.3}_acc-{:.3}.pth'.format(epoch_num, val_loss, val_acc)
            path_to_saving_model = os.path.join(path_to_weights, model_name)
            torch.save(mfcc_emotion_cnn.state_dict(), path_to_saving_model)
            print('model %s have been saved' % (path_to_saving_model))

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        train_loss_list.append(epoch_loss)
        val_loss_list.append(val_loss)
            
    return model, train_acc_list, val_acc_list, train_loss_list, val_loss_list

In [14]:
path_to_csv = os.path.join('/media/mikhail/files/datasets/emotion_recognition/CREMA-D', 'VideoDemographics.csv')
demographic_info = pd.read_csv(path_to_csv)
gender_df = demographic_info[['ActorID','Sex']]

batch_size=256


# CREMA-D
target_path = '/media/mikhail/files/datasets/emotion_recognition/CREMA-D/AudioWAV'
# IEMOCAP
#target_path = '/media/mikhail/files/datasets/emotion_recognition/IEMOCAP/IEMOCAP_full_release/audios'
 
npys_list = get_paths_to_wavs(target_path)

# shuffle the dataset to for the learning process stability
random.seed(0)
random.shuffle(npys_list)

dataset_size = len(npys_list)

train_size = int(0.8 * dataset_size)

print(dataset_size)

train_dataset = crema_gender_dataset(npys_list[:train_size], (256, 256), 'TRAIN', gender_df)
#train_dataset = numpy_iemocap_dataset(npys_list[:train_size], (256, 256), mode='TRAIN')

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# set up test dataset and test dataloader
test_dataset = crema_gender_dataset(npys_list[train_size:], (256, 256), 'TEST', gender_df)
#test_dataset = numpy_iemocap_dataset(npys_list[train_size:], (256, 256), mode='TEST')

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

7442


AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [16]:
# set-up devices
cuda = torch.device('cuda:0')
cpu = torch.device('cpu')

#model = audio_cnn(rows=256, cols=256, num_classes=len(train_dataset.emotions_dict))
model = models.alexnet()
model.features[0] = nn.Conv2d(1,  64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
model.classifier[4] = nn.Linear(4096, 1024)
model.classifier[6] = nn.Linear(1024, 256)
model.classifier.add_module('7', nn.ReLU(inplace=True))
model.classifier.add_module('8', nn.Dropout())
model.classifier.add_module('9', nn.Linear(256, len(train_dataset.emotions_dict)))

'''
model = models.vgg16_bn()
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=len(train_dataset.emotions_dict), bias=True)
)


model = models.resnet34()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Sequential(
    nn.Linear(512, 128),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(128, len(train_dataset.emotions_dict))
)


model = models.resnext50_32x4d()
model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(128, len(train_dataset.emotions_dict))
)


model = models.mobilenet_v2()
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.classifier = nn.Sequential(
    nn.Linear(1280, 512),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(128, len(train_dataset.emotions_dict))
)
'''


model(torch.randn((1, 1, 256, 256)))

summary(model, input_size=(1, 299, 299), batch_size=batch_size, device='cpu')

device = cuda

model.to(device)

criterion = nn.CrossEntropyLoss()
# define an optimization algorithm and bind it with the NN parameters
optimizer = torch.optim.Adam(params=model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


starting_epoch = 0
ending_epoch = 1000
epoch_step = 1

basic_name = '{}_log_pow_spec_norm_alexnet_lr_decay'.format('CREMA')



path_to_weights = basic_name
path_to_pkl = basic_name

if not os.path.isdir(path_to_weights):
    os.mkdir(path_to_weights)
if not os.path.isdir(path_to_pkl):
    os.mkdir(path_to_pkl)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [256, 64, 74, 74]           7,808
              ReLU-2          [256, 64, 74, 74]               0
         MaxPool2d-3          [256, 64, 36, 36]               0
            Conv2d-4         [256, 192, 36, 36]         307,392
              ReLU-5         [256, 192, 36, 36]               0
         MaxPool2d-6         [256, 192, 17, 17]               0
            Conv2d-7         [256, 384, 17, 17]         663,936
              ReLU-8         [256, 384, 17, 17]               0
            Conv2d-9         [256, 256, 17, 17]         884,992
             ReLU-10         [256, 256, 17, 17]               0
           Conv2d-11         [256, 256, 17, 17]         590,080
             ReLU-12         [256, 256, 17, 17]               0
        MaxPool2d-13           [256, 256, 8, 8]               0
AdaptiveAvgPool2d-14           [256, 25

In [17]:
start_epoch = 0
epochs = 500
epoch_step = 1

print('Start learning')


best_acc = 0.0

train_dataset_size = len(train_dataloader.dataset)  
test_dataset_size = len(test_dataloader.dataset)  


if os.path.exists(os.path.join(path_to_pkl, basic_name + '_train_loss.pkl')):
    # Update existing classifier
    with open(os.path.join(path_to_pkl, basic_name + '_train_loss.pkl'), "rb") as f:
        train_loss_list = pickle.load(f)
else:
  train_loss_list = []

if os.path.exists(os.path.join(path_to_pkl, basic_name + '_train_acc.pkl')):
    # Update existing classifier
    with open(os.path.join(path_to_pkl, basic_name + '_train_acc.pkl'), "rb") as f:
        train_acc_list = pickle.load(f)
else:
  train_acc_list = []

if os.path.exists(os.path.join(path_to_pkl, basic_name + '_val_loss.pkl')):
    # Update existing classifier
    with open(os.path.join(path_to_pkl, basic_name + '_val_loss.pkl'), "rb") as f:
        val_loss_list = pickle.load(f)
else:
  val_loss_list = []

if os.path.exists(os.path.join(path_to_pkl, basic_name + '_val_acc.pkl')):
    # Update existing classifier
    with open(os.path.join(path_to_pkl, basic_name + '_val_acc.pkl'), "rb") as f:
        val_acc_list = pickle.load(f)
else:
    val_acc_list = []


t = 0.0

for epoch_idx in range(start_epoch, epochs, epoch_step):

    print('#############################################')
    print('#\tStart training process')
    print('#############################################\n\n')

    # iterate over epochs
    for epoch in range(epoch_step):
        print('Epoch #{}'.format(epoch_idx + epoch))
        t0 = time.time()
        model.train()
        # define losses and correct valuse number for each epoch
        epoch_train_loss = 0.0
        correct = 0
        total = 0
        
        # iterate over batches
        for data, labels in train_dataloader:
            data = data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            pred = model(data)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item() * data.size(0)
            total += labels.size(0)
            _, pred_labels = torch.max(pred.data, 1)

            correct += (pred_labels == labels).sum().item()

        t1 = time.time()

        print('Epoch time = {:.3f} s'.format(t1 - t0))

        train_loss = epoch_train_loss / train_dataset_size
        train_acc = correct/total

        train_acc_list.append(train_acc)
        train_loss_list.append(train_loss)

        print('Loss = %f\tTraining acc = %f' % (train_loss, train_acc))
        print('----------------------------------------')
        
        print('#############################################')
        print('#\tStart validation on %d epoch' % (epoch_idx + epoch))
        print('#############################################')
        
        model.eval()
        with torch.no_grad():
            true_values = 0.0
            epoch_test_loss = 0.0
            correct = 0
            total = 0
            for data, labels in test_dataloader:
                data = data.to(device)
                labels = labels.to(device)
                # run the model
                pred = model(data)
                loss = criterion(pred, labels)
                epoch_test_loss += loss.item() * data.size(0)
                total += labels.size(0)
                _, pred_labels = torch.max(pred.data, 1)
                correct += (pred_labels == labels).sum().item()
        val_acc = correct/total
        val_loss = epoch_test_loss / test_dataset_size
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
                
        print('\tLoss = {:.4f}\tValidation acc = {:.3f}'.format(val_loss, val_acc))
        print('---------------------------------------------')

        scheduler.step()
      
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #save current model to resume training
        #после каждой эпохи сохраняем веса для того, чтобы потом продолжить обучение именно с последней эпохи
        #а не с эпохи с лучшими весами 
        path_to_saving_model = os.path.join(path_to_weights, basic_name + '_current.pth')
        torch.save(model.state_dict(), path_to_saving_model)
            
        if val_acc > best_acc:
            print('#############################################')
            print('#\tBest accuracy has achieved')
            print('#\tSaving weights...')
            print('#############################################\n\n')

            model_name = basic_name + '_ep-{}_loss-{:.3}_acc-{:.3}.pth'.format(epoch_idx + epoch, val_loss, val_acc)
            path_to_saving_model = os.path.join(path_to_weights, model_name)

            torch.save(model.state_dict(), path_to_saving_model)
            print('model {} have been saved'.format(path_to_saving_model))
            best_acc = val_acc

        with open(os.path.join(path_to_pkl, basic_name + '_train_loss.pkl'), 'wb') as f:
            pickle.dump(train_loss_list, f)

        with open(os.path.join(path_to_pkl, basic_name + '_train_acc.pkl'), 'wb') as f:
            pickle.dump(train_acc_list, f)

        with open(os.path.join(path_to_pkl, basic_name + '_val_loss.pkl'), 'wb') as f:
            pickle.dump(val_loss_list, f)

        with open(os.path.join(path_to_pkl, basic_name + '_val_acc.pkl'), 'wb') as f:
            pickle.dump(val_acc_list, f)
    

 validation on 454 epoch
#############################################
	Loss = 1.2513	Validation acc = 0.555
---------------------------------------------
#############################################
#	Start training process
#############################################


Epoch #455
Epoch time = 41.285 s
Loss = 1.063009	Training acc = 0.609777
----------------------------------------
#############################################
#	Start validation on 455 epoch
#############################################
	Loss = 1.2513	Validation acc = 0.555
---------------------------------------------
#############################################
#	Start training process
#############################################


Epoch #456
Epoch time = 42.407 s
Loss = 1.054113	Training acc = 0.614312
----------------------------------------
#############################################
#	Start validation on 456 epoch
#############################################
	Loss = 1.2513	Validation acc = 0.555
---------